# Processing and Plotting SeaBASS (.sb) Files

This notebook contains a workflow for reading multiple SeaBASS (`.sb`) files from a folder, combining them, subsetting the data by date and station, and then plotting specific parameters.

We will use:
* `SB_support.py` (the provided module) to read the files.
* `pandas` to aggregate and filter the data.
* `matplotlib` to plot the results.

# Workflow:

The notebook:
1.  Loads all `.sb` files from the target directory.
2.  Parses them using the `readSB` class.
3.  Combines them into a single `pandas.DataFrame`.
4.  Filteres the data based on your specified `station` and `datetime` range.
5.  Plots the requested parameters (`ag`, `ap`, `ad`) over time.

In [1]:
import os
import tarfile
import glob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime
import re  # Import regular expressions
import folium
from folium.plugins import MarkerCluster

# Import the readSB class from the local .py file
try:
    from SB_support import readSB
    print("SB_support module loaded successfully.")
except ImportError:
    print("ERROR: Could not find SB_support.py.")
    print("Please make sure SB_support.py is in the same directory as this notebook.")

# Set up plotting style
plt.style.use('ggplot')
%matplotlib inline

SB_support module loaded successfully.


### Decompress Archive and List .sb Files

In [ ]:
# --- Configuration ---
SOURCE_FOLDER = 'seabass_data'
DECOMPRESSED_FOLDER = 'seabass_data_decompressed'
# ---------------------

print(f"Searching for .tar archives in '{SOURCE_FOLDER}'...")

# Find any tar files (common extensions)
search_paths = [
    os.path.join(SOURCE_FOLDER, '*.tar'),
    os.path.join(SOURCE_FOLDER, '*.tar.gz'),
    os.path.join(SOURCE_FOLDER, '*.tgz')
]

tar_files_found = []
for path in search_paths:
    tar_files_found.extend(glob.glob(path))

if not tar_files_found:
    print(f"ERROR: No .tar, .tar.gz, or .tgz files found in '{SOURCE_FOLDER}'.")
else:
    # Use the first archive found
    tar_file_path = tar_files_found[0]
    print(f"Found archive: {tar_file_path}")

    # Create the output directory if it doesn't exist
    os.makedirs(DECOMPRESSED_FOLDER, exist_ok=True)
    
    extracted_filenames = []
    
    try:
        # Open the tar file with auto-compression detection ('r:*')
        with tarfile.open(tar_file_path, 'r:*') as tar:
            
            print(f"Extracting '.sb' files to '{DECOMPRESSED_FOLDER}'...")
            
            # Find and extract only members that are .sb files
            for member in tar.getmembers():
                if member.name.endswith('.sb') and member.isfile():
                    tar.extract(member, path=DECOMPRESSED_FOLDER)
                    # Store just the base name of the file
                    extracted_filenames.append(os.path.basename(member.name))
        
        if not extracted_filenames:
            print("No '.sb' files were found inside the archive.")
        else:
            print(f"\nSuccessfully extracted {len(extracted_filenames)} .sb files.")
            
            # --- Print the first 10 filenames ---
            print("\nFirst 10 extracted filenames:")
            for filename in extracted_filenames[:10]:
                print(f"  -> {filename}")

    except tarfile.ReadError:
        print(f"ERROR: File is not a valid tar archive: {tar_file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

### Read the .sb file and Plot Spectral Data (Wavelength vs. Parameter)

In [ ]:
# --- Configuration ---
DECOMPRESSED_FOLDER = 'seabass_data_decompressed'
PARAMETER_TO_PLOT = 'aph'
TARGET_CRUISE = 'WS25024'
TARGET_STATION = 'ALL' # Set to 'ALL', None, or a specific station like 'WS'
# ---------------------

# --- 1. Find all .sb files ---
search_path = os.path.join(DECOMPRESSED_FOLDER, '**', '*.sb')
all_sb_files = glob.glob(search_path, recursive=True)

# --- List to hold data for the *next* cell ---
files_to_map = [] 
# ---------------------------------------------

files_to_process = []
station_labels = []

print(f"Found {len(all_sb_files)} total .sb files. Filtering based on headers...")

# --- 2. Filter files by reading metadata ---
for f_path in all_sb_files:
    try:
        sb_data = readSB(filename=f_path, no_warn=True)
        
        # --- Apply Filters ---
        if sb_data.headers.get('cruise') != TARGET_CRUISE:
            continue
            
        file_station = sb_data.headers.get('station', 'unknown')
        if TARGET_STATION is not None and TARGET_STATION.upper() != 'ALL':
            if file_station != TARGET_STATION:
                continue
                
        if 'wavelength' not in sb_data.data or PARAMETER_TO_PLOT not in sb_data.data:
            continue
            
        # --- If all checks pass, save file for plotting ---
        files_to_process.append(f_path)
        station_labels.append(file_station)
        
        # --- Extract and save coordinate data for mapping ---
        lat_str = sb_data.headers.get('north_latitude', None)
        lon_str = sb_data.headers.get('east_longitude', None)
        
        if lat_str and lon_str:
            try:
                lat = float(lat_str.replace('[DEG]', ''))
                lon = float(lon_str.replace('[DEG]', ''))
                # Add this file's info to our map list
                files_to_map.append({
                    'station': file_station,
                    'lat': lat,
                    'lon': lon,
                    'file': os.path.basename(f_path)
                })
            except (ValueError, TypeError):
                print(f"  -> Warning: Could not parse coordinates for {os.path.basename(f_path)}. Will not be mapped.")
        
    except Exception as e:
        print(f"  -> Warning: Skipped {os.path.basename(f_path)} due to read error: {e}")

# --- 3. Plot the filtered files ---
if not files_to_process:
    print(f"\nNo files matched your filter criteria. Nothing to plot.")
    # ... (rest of error printing) ...
else:
    print(f"\nFound {len(files_to_process)} files to plot. Reading and plotting...")
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    for f_path, station_label in zip(files_to_process, station_labels):
        # ... (rest of plotting code is identical) ...
        try:
            sb_data = readSB(filename=f_path, no_warn=True)
            df = pd.DataFrame(sb_data.data)
            df['wavelength'] = pd.to_numeric(df['wavelength'], errors='coerce')
            df[PARAMETER_TO_PLOT] = pd.to_numeric(df[PARAMETER_TO_PLOT], errors='coerce')
            df_clean = df.dropna(subset=['wavelength', PARAMETER_TO_PLOT])
            df_sorted = df_clean.sort_values(by='wavelength')
            ax.plot(df_sorted['wavelength'], df_sorted[PARAMETER_TO_PLOT], 
                    label=f"St. {station_label}", 
                    alpha=0.7)
        except Exception as e:
            print(f"  -> ERROR plotting {os.path.basename(f_path)}: {e}")

    # --- 4. Finalize Plot ---
    plot_title = f"{PARAMETER_TO_PLOT.upper()} Spectra for Cruise '{TARGET_CRUISE}'"
    if TARGET_STATION is not None and TARGET_STATION.upper() != 'ALL':
        plot_title += f" (Station: {TARGET_STATION})"
    
    ax.set_title(plot_title)
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel(f"{PARAMETER_TO_PLOT} (1/m)")
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Stations")
    ax.grid(True)
    plt.show()

### Check depth distribution of ap or ad parameters in .sb files 

In [ ]:
depth_values = []
files_with_depth = 0
files_without_depth = 0

print(f"Scanning {len(all_sb_files)} files for 'measurement_depth'...")

# 2. Extract measurement_depth from each file
for f_path in all_sb_files:
    try:
        # Read the file headers only (we don't need the full data matrix here)
        sb_data = readSB(filename=f_path, no_warn=True)
        
        # Extract the depth
        depth_str = sb_data.headers.get('measurement_depth', None)
        
        if depth_str:
            try:
                # Remove any units like [m] if present and convert to float
                clean_depth = float(str(depth_str).replace('[m]', '').replace('[M]', ''))
                depth_values.append(clean_depth)
                files_with_depth += 1
            except ValueError:
                # Handle cases where depth might be non-numeric
                files_without_depth += 1
        else:
            files_without_depth += 1
            
    except Exception as e:
        print(f"  -> Warning: Error reading {os.path.basename(f_path)}: {e}")

# 3. Plot the Histogram
if not depth_values:
    print("No valid 'measurement_depth' values found in any files.")
else:
    print(f"\nFound depth values in {files_with_depth} files.")
    print(f"Skipped {files_without_depth} files (missing or invalid depth).")
    
    # Calculate some basic stats
    min_depth = min(depth_values)
    max_depth = max(depth_values)
    avg_depth = np.mean(depth_values)
    
    print(f"Depth Range: {min_depth}m to {max_depth}m")
    print(f"Average Depth: {avg_depth:.2f}m")

    plt.figure(figsize=(10, 6))
    
    # Create the histogram
    # 'bins' determines how many bars. 'auto' usually works well.
    plt.hist(depth_values, bins='auto', color='skyblue', edgecolor='black', alpha=0.7)
    
    plt.title("Distribution of Measurement Depths Across All Files")
    plt.xlabel("Measurement Depth (m)")
    plt.ylabel("Number of Files")
    plt.grid(axis='y', alpha=0.5)
    
    plt.show()

### Show the location of the sampling station

In [ ]:
print("--- Generating Map ---")

# Check if the 'files_to_map' list was created by the previous cell
if 'files_to_map' not in locals() or not files_to_map:
    print("ERROR: 'files_to_map' list not found or is empty.")
    print("Please (re)run the previous cell (Cell 3) to filter and collect data first.")
else:
    print(f"Found {len(files_to_map)} stations to map.")
    
    # Calculate the center of the map
    lats = [item['lat'] for item in files_to_map]
    lons = [item['lon'] for item in files_to_map]
    avg_lat = sum(lats) / len(lats)
    avg_lon = sum(lons) / len(lons)

    # Create the map, centered on the average location
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)
    
    # Create a cluster group to hold the markers
    marker_cluster = MarkerCluster().add_to(m)

    # Add a marker for each file found
    for item in files_to_map:
        
        # Create popup message
        popup_html = f"""
        <b>Station:</b> {item['station']}<br>
        <b>File:</b> {item['file']}<br>
        <b>Coords:</b> {item['lat']}, {item['lon']}
        """

        # Add a marker to the *cluster*
        folium.Marker(
            [item['lat'], item['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"Station {item['station']}"
        ).add_to(marker_cluster)

    # Display the map
    print("Displaying map with all filtered stations...")
    display(m)

### Load SFER_data.csv and Create Final Table

In [2]:
# --- Configuration (Using variables from previous cells for consistency) ---
DECOMPRESSED_FOLDER = 'seabass_data_decompressed'
SFER_FILE = 'SFER_data.csv'

# New Filters and Extraction Targets
# This is now the CLASSIFICATION THRESHOLD: >= 5.0m is 'Bottom'
MIN_MEASUREMENT_DEPTH = 5.0 
TARGET_WAVELENGTH = 443
PARAMETER_TO_EXTRACT = 'ag'
TARGET_CRUISE = 'ALL' # 'ALL' is maintained
# --------------------------------------------------------------------------

# --- 1. Prepare SFER_data.csv (Lookup Table) ---
if not os.path.exists(SFER_FILE):
    print(f"ERROR: File not found: {SFER_FILE}")
    raise FileNotFoundError(SFER_FILE)

print(f"Loading {SFER_FILE} and preparing for depth matching...")
sfer_df = pd.read_csv(SFER_FILE, low_memory=False)

# Prepare SFER_data for merging
sfer_df['datetime'] = pd.to_datetime(sfer_df['datetime'], errors='coerce')
sfer_df.rename(columns={'datetime': 'datetime_sfer'}, inplace=True) # Rename SFER datetime
sfer_df['station'] = sfer_df['station'].astype(str)
sfer_df['cruise_id'] = sfer_df['cruise_id'].astype(str) # Ensure cruise_id is string

# NOTE: No filtering by depth_class is done here; all SFER depths are kept for merging.
print(f"SFER_data has {len(sfer_df)} total records.")

# --- 2. Find, Filter, and Extract Data from .sb files ---
search_path = os.path.join(DECOMPRESSED_FOLDER, '**', '*.sb')
all_sb_files = glob.glob(search_path, recursive=True)
extracted_data_list = []

print(f"Scanning {len(all_sb_files)} .sb files and classifying depth (>= {MIN_MEASUREMENT_DEPTH}m is 'Bottom')...")

for f_path in all_sb_files:
    try:
        sb_data = readSB(filename=f_path, no_warn=True)
        
        # --- Apply Metadata Filters ---
        file_cruise = sb_data.headers.get('cruise', 'unknown')
        if TARGET_CRUISE.upper() != 'ALL': 
            if file_cruise != TARGET_CRUISE:
                continue

        # Filter 1: Check fields exist
        if 'wavelength' not in sb_data.data or PARAMETER_TO_EXTRACT not in sb_data.data:
            continue
            
        # --- CLASSIFICATION AND EXTRACTION ---
        
        # A) Determine depth class
        depth_str = sb_data.headers.get('measurement_depth', None)
        
        if depth_str:
            measurement_depth = float(depth_str.replace('[m]', '').replace('[M]', ''))
            # Derive depth_class based on the threshold
            depth_class_sb = 'Bottom' if measurement_depth >= MIN_MEASUREMENT_DEPTH else 'Surface'
        else:
            # If depth is missing, we classify it as 'Unknown' so it won't match 'Surface' or 'Bottom'
            depth_class_sb = 'Unknown' 
            
        # B) Extract remaining metadata
        file_station = sb_data.headers.get('station', 'unknown')
        lat_str = sb_data.headers.get('north_latitude', None)
        lon_str = sb_data.headers.get('east_longitude', None)
        dt_list = sb_data.fd_datetime()
        
        dt_object = dt_list[0] if dt_list else None
        lat_float = float(lat_str.replace('[DEG]', '')) if lat_str else np.nan
        lon_float = float(lon_str.replace('[DEG]', '')) if lon_str else np.nan
        
        # C) Extract target value (e.g., ag at 443nm)
        df = pd.DataFrame(sb_data.data)
        target_row = df[pd.to_numeric(df['wavelength'], errors='coerce') == TARGET_WAVELENGTH]
        param_value = target_row[PARAMETER_TO_EXTRACT].values[0] if not target_row.empty else np.nan
            
        # D) Add all data to our list
        extracted_data_list.append({
            'cruise_id': file_cruise,
            'station_id': file_station,
            'latitude': lat_float,
            'longitude': lon_float,
            'datetime_sb': dt_object,
            'depth_class_sb': depth_class_sb, # NEW: Derived depth class
            f'{PARAMETER_TO_EXTRACT}_{TARGET_WAVELENGTH}': param_value,
            'original_filename': os.path.basename(f_path)
        })
        
    except Exception as e:
        pass # Continue processing other files

# --- 3. Create the main DataFrame from the .sb files ---
if not extracted_data_list:
    print("\nNo files matched the filter criteria. No data to merge.")
else:
    sb_df = pd.DataFrame(extracted_data_list)
    print(f"\nExtracted data from {len(sb_df)} .sb files.")
    
    # Prepare sb_df for merging
    sb_df['station_id'] = sb_df['station_id'].astype(str)
    sb_df['cruise_id'] = sb_df['cruise_id'].astype(str)

    # --- 4. Perform Left Merge for Diagnosis ---
    if not os.path.exists(SFER_FILE):
        print(f"ERROR: File not found: {SFER_FILE}")
    else:
        try:
            print("Performing diagnostic merge (Left Join)...")
            
            # --- MERGE ON CRUISE, STATION, AND DERIVED DEPTH CLASS ---
            # Merge is on Cruise ID, Station ID, and the derived depth class.
            merged_df = pd.merge(
                sb_df,
                sfer_df, # Merge against the full SFER DF (with datetime_sfer)
                left_on=['cruise_id', 'station_id', 'depth_class_sb'],
                right_on=['cruise_id', 'station', 'depth_class'], # match derived SB depth_class with SFER depth_class
                how='left',
                indicator=True # Creates a column '_merge' showing if match was found
            )

            # 5. --- DIAGNOSTIC: FIND UNMATCHED FILES ---
            unmatched_files = merged_df[merged_df['_merge'] == 'left_only']
            
            print(f"\n--- Diagnostic Results ({len(unmatched_files)} Unmatched Files) ---")
            if not unmatched_files.empty:
                print("These files from the .sb set failed to find a match in SFER_data.csv:")
                unmatched_files_report = unmatched_files[['original_filename', 'station_id', 'depth_class_sb']].copy()
                
                print(f"Found {len(unmatched_files_report)} total unmatched records.")
                print(f"Example of unmatched records (first 10):")
                # Sort and print the unique filenames
                unmatched_report = unmatched_files[['original_filename', 'cruise_id', 'station_id', 'depth_class_sb']].head(10).to_markdown(index=False)
                print(unmatched_report)

            else:
                print("All extracted .sb files matched a record in the SFER_data.csv!")

            # 6. --- MAIN MERGE (Inner Join) for final table ---
            # Perform the inner join again for the final clean table
            final_merged_df = merged_df[merged_df['_merge'] == 'both'].copy()

            final_column_order = [
                'keyfield',
                'cruise_id',
                'station_id',
                'latitude',
                'longitude',
                'datetime_sfer', 
                'datetime_sb',   
                'depth_class', # Original SFER column, now matched
                f'{PARAMETER_TO_EXTRACT}_{TARGET_WAVELENGTH}',
                'original_filename'
            ]
            
            # Select and reorder
            cols_to_keep = [col for col in final_column_order if col in final_merged_df.columns]
            final_table = final_merged_df[cols_to_keep]

            # 7. Display the final table
            print("\n--- Final Table (Inner Merge) ---")
            print(f"Result has {len(final_table)} merged rows.")
            
            try:
                print(final_table.to_markdown(index=False))
            except ImportError:
                 print("Error: The 'tabulate' library is missing. Please run 'pip install tabulate' to view the table.")


        except Exception as e:
            print(f"An error occurred during the merge process: {e}")

Loading SFER_data.csv and preparing for depth matching...
SFER_data has 9300 total records.
Scanning 5381 .sb files and classifying depth (>= 5.0m is 'Bottom')...

Extracted data from 2319 .sb files.
Performing diagnostic merge (Left Join)...

--- Diagnostic Results (339 Unmatched Files) ---
These files from the .sb set failed to find a match in SFER_data.csv:
Found 339 total unmatched records.
Example of unmatched records (first 10):
| original_filename                                   | cruise_id   | station_id   | depth_class_sb   |
|:----------------------------------------------------|:------------|:-------------|:-----------------|
| SFMBON_WS16130_ag_LK_1m_018_201605101310_R3.sb      | WS16130     | LK           | Surface          |
| SFMBON_WS16130_ag_LK_29m_024_201605101310_R3.sb     | WS16130     | LK           | Bottom           |
| SFMBON_WS21151_ag_LK_1m_033_202106010328_R3.sb      | WS21151     | LK           | Surface          |
| SFMBON_WS21151_ag_LK_35m_037_2021060103

### Correct cruise and station ID's of unmatched .sb files and remerge records with SFER_data for surface samples

In [ ]:
# --- Correction & Remerging Cell ---

# --- FILE CORRECTIONS:
# 'SFMBON_WS21093_ag_49_1m_105_202104050623_R1.sb' is not included because station 49 does not exist in SFER_data.csv.

# 1. Define Correction Maps
#    Add any unmatched IDs you found here.
#    Format: { 'ID_in_SeaBASS_file' : 'ID_in_SFER_CSV' }
station_corrections = {
    'LK': '21LK',       # Fixes station LK -> 21LK
    '57.0': '57',  
    'kw2': 'KW2',
    'cal1': 'CAL1',
    'cal3': 'CAL3',
    'cal5': 'CAL5',
    'Cal1': 'CAL1',
    'Cal3': 'CAL3',
    'Cal5': 'CAL5'
}

cruise_corrections = {
    'H23138': 'HG23138', 
    'WS20278': 'WS20279',
    'WS21277': 'WS21278',
    'H23314': 'HG23314'
}

# Check if 'sb_df' exists from the previous cell
if 'sb_df' not in locals():
    print("ERROR: 'sb_df' dataframe not found. Please run the extraction cell first.")
else:
    print("Applying ID corrections to SeaBASS data...")

    # 2. Apply Corrections to sb_df
    # .replace() looks for the keys in the dict and swaps them with values
    sb_df['station_id'] = sb_df['station_id'].replace(station_corrections)
    sb_df['cruise_id'] = sb_df['cruise_id'].replace(cruise_corrections)

    # 3. Re-Merge with SFER Data
    #    We use the already loaded 'sfer_df' if available, or reload it.
    if 'sfer_df' not in locals():
        if os.path.exists(SFER_FILE):
            sfer_df = pd.read_csv(SFER_FILE, low_memory=False)
            if 'datetime' in sfer_df.columns:
                sfer_df.rename(columns={'datetime': 'datetime_sfer'}, inplace=True)
            sfer_df['station'] = sfer_df['station'].astype(str)
            sfer_df['cruise_id'] = sfer_df['cruise_id'].astype(str)
        else:
            print(f"ERROR: {SFER_FILE} not found.")
            sfer_df = pd.DataFrame()

    if not sfer_df.empty:
        print("Re-merging data with corrected IDs...")
        
        # Perform the merge again
        final_merged_df = pd.merge(
            sb_df,
            sfer_df,
            left_on=['cruise_id', 'station_id','depth_class_sb'],
            right_on=['cruise_id', 'station','depth_class'],
            how='left',
            indicator=True # Creates a column '_merge' showing if match was found
        )

        # 4. Select and Order Columns
        #    Define the columns we want in the final output
        target_columns = [
            'keyfield',
            'cruise_id',
            'station_id',
            'latitude',
            'longitude',
            'datetime_sfer',
            'datetime_sb',
            f'{PARAMETER_TO_EXTRACT}_{TARGET_WAVELENGTH}', # e.g. ag_443
            'original_filename'
        ]
        
        # Filter for columns that actually exist to avoid errors
        existing_cols = [c for c in target_columns if c in final_merged_df.columns]
        final_table_corrected = final_merged_df[existing_cols]

        # 5. --- DIAGNOSTIC: FIND UNMATCHED FILES ---
        unmatched_files_final = final_merged_df[final_merged_df['_merge'] == 'left_only']
            
        print(f"\n--- Diagnostic Results ({len(unmatched_files_final)} Unmatched Files) ---")
        if not unmatched_files_final.empty:
            print("These files from the .sb set failed to find a match in SFER_data.csv:")
            unmatched_files_report_final = unmatched_files_final[['original_filename', 'station_id', 'depth_class_sb']].copy()
                
            print(f"Found {len(unmatched_files_report_final)} total unmatched records.")
            print(f"Example of unmatched records (first 20):")
            # Sort and print the unique filenames
            unmatched_report_final = unmatched_files_final[['original_filename', 'cruise_id', 'station_id', 'depth_class_sb']].head(20).to_markdown(index=False)
            print(unmatched_report_final)
        else:
            print("\n✅ SUCCESS: All records matched successfully!")

        # 6. Sort final table by datetime_sfer
        if 'datetime_sfer' in final_table_corrected.columns:
            final_table_corrected = final_table_corrected.sort_values(by='datetime_sfer')

        # 7. Display Results (Cleaned)
        print("\n--- Corrected Final Table (First 5 Rows) ---")
        print(final_table_corrected.head().to_markdown(index=False))

        # 8. Save the corrected file
        final_table_corrected.to_csv('seabass_sfer_merged.tsv', sep='\t', index=False)
        print(f"\nSaved cleaned table to: {'seabass_sfer_merged'}")

Applying ID corrections to SeaBASS data...
Re-merging data with corrected IDs...

--- Diagnostic Results (18 Unmatched Files) ---
These files from the .sb set failed to find a match in SFER_data.csv:
Found 18 total unmatched records.
Example of unmatched records (first 20):
| original_filename                                    | cruise_id   | station_id   | depth_class_sb   |
|:-----------------------------------------------------|:------------|:-------------|:-----------------|
| SFMBON_WS25024_ag_REDTIDE_1m_268_202501262126_R1.sb  | WS25024     | REDTIDE      | Surface          |
| SFMBON_WS21093_ag_49_1m_108_202104050623_R3.sb       | WS21093     | 49           | Surface          |
| SFMBON_WS18120_ag_68_5m_048_201805031110_R3.sb       | WS18120     | 68           | Bottom           |
| SFMBON_WS19210_ag_9B_2m_048_201907300047_R3.sb       | WS19210     | 9B           | Surface          |
| SFMBON_WS19119_ag_CBH_30m_160_201905012017_R3.sb     | WS19119     | CBH          | Bottom   